# X-ray Absorption Spectroscopy (XAS) Experiment at SCS

Author: Jun Zhu jun.zhu@xfel.eu

Created on Nov. 12, 2018

Copyright (C) European X-Ray Free-Electron Laser Facility GmbH.

All rights reserved.

## Introduction

<table><tr>
    <td>
        <img src='misc/mono.png' width="600 px">
        <figcaption>Higley et al., Rev. Sci. Instrum.  87, 033110 (2016)</figcaption>
    </td>
    <td>
        <img src='misc/illustration.png' width="920 px">
    </td>
    <td>
        <img src='misc/expected_data.png' width="800 px">
        <figcaption>Higley et al., submitted</figcaption></td>
</tr></table>

## Data Analysis

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from karabo_data import RunDirectory

In [ ]:
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.figsize'] = (8.0, 6.0)

In [ ]:
!ls /gpfs/exfel/exp/SCS/201831/p900048/raw

In [ ]:
run_folder = "/gpfs/exfel/exp/SCS/201831/p900048/raw/r0150"
run = RunDirectory(run_folder)
run.info()

## Sanity check

*check whether the pulse energy in the pipeline data is calibrated*

In [ ]:
xgm_device = 'SCS_BLU_XGM/XGM/DOOCS'
# xgm_device = 'SA3_XTD10_XGM/XGM/DOOCS'

adc_device = 'SCS_UTC1_MCP/ADC/1'
adc_channels = ["{}:channel_{}.output".format(adc_device, ch) for ch in [9, 8, 7, 6]]
adc_channel_names = ["MCP1", "MCP2", "MCP3", "MCP4"]

In [ ]:
idx = -1
tid, data = run.train_from_index(idx)

In [ ]:
xgm_run = run.get_dataframe(fields=[(xgm_device, '*value')])
xgm_run.rename(columns=lambda x: x.split('/')[-1], inplace=True)

# pulse_energy_train_resolved = xgm_run['pulseEnergy.photonFlux'].iloc[idx]
# pulse_energy_pulse_resolved = data[xgm_device + ':output']['data.intensityTD'][
#     0:int(xgm_run['pulseEnergy.nummberOfBrunches'].iloc[idx])].mean()

# print("Train resolved pulse energy is: {:.4f} micro J".format(pulse_energy_train_resolved))
# print("Pulse resolved pulse energy is: {:.4f} micro J".format(pulse_energy_pulse_resolved))
# if abs(pulse_energy_train_resolved - pulse_energy_pulse_resolved) > 0.01*pulse_energy_train_resolved:
#     print("Warning: pulse resolved pulse energy is not calibrated!")

***check the numbers of pulses and train resolved pulse energies throughtout the run***

In [ ]:
_, ax = plt.subplots(figsize=(12, 4.5))
ax2 = ax.twinx()

ln1 = ax.plot(xgm_run['pulseEnergy.nummberOfBrunches'], label="number of pulses", c='g')
ln2 = ax2.plot(xgm_run['pulseEnergy.photonFlux'], label=r"pulse energy ($\mu$J)")

lns = ln1 + ln2
lables = [l.get_label() for l in lns]
ax.legend(lns, lables, loc=0)
ax.set_xlabel("Train ID")
ax.set_ylabel("Number of pulses")
ax2.set_ylabel(r"Pulse energy ($\mu$J)")

plt.tight_layout()

***check that the interested FastAdc channels all have data***

In [ ]:
import re


working_adc_channels = []
for key, value in sorted(data.items()):
    if re.search(re.compile(r"{}.*output".format(adc_device)), key) and 'data.rawData' in value:
        working_adc_channels.append(key)
        
for ch in adc_channels:
    if ch not in working_adc_channels:
        raise ValueError("{} is not a working FastAdc channel!".format(ch))

In [ ]:
fig, axes = plt.subplots(len(adc_channels), 1, figsize=(14, 6*len(adc_channels)))

for ax, channel in zip(axes, adc_channels):
    ax.plot(data[adc_channels[2]]['data.rawData'] - data[adc_channels[2]]['data.baseline'], '.-')
    ax.set_title(channel)
    ax.set_ylabel("intensity (arb.)")

ax.set_xlabel("samples")
ax.set_xlim(1500, 1700)

print("Trigger time: {}".format(data[adc_device]['triggerTime.value']))
print("Delay: {}".format(data[adc_device]['delay.value']))

## Data analysis

In [ ]:
def integrate_adc_channel(run, ch, threshold=-200):
    """Integration along a FastAdc channel.
    
    :param DataCollection run: run data. 
    :param str ch: output channel name.
    :param int threshold: data above this threshold will be ignored.
    
    :return
    """
    background = run.get_array(ch, 'data.baseline')
    raw_data = run.get_array(ch, 'data.rawData') - background

    return np.trapz(raw_data.where(raw_data < threshold, 0), axis=1)


def process_run(run_directory, xgm_id, adc_id, adc_channels):
    """Process the run data.
    
    :param DataCollection run: run data. 
    :param str xgm_id: XgmDoocs device ID.
    :param str adc_id: FastAdc device ID.
    :param list adc_channels: names of FastAdc channels which received data.
    
    :return
    """    
    xgm = run.get_array(xgm_id + ":output", 'data.intensityTD').max(axis=1)
    
    mcps = []
    for ch in adc_channels:
        mcps.append(integrate_adc_channel(run, ch))
        
    return xgm, mcps


ret = process_run(run_folder, xgm_device, adc_device, adc_channels)

#### Correlation between I0 and I1

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

for i, ax in enumerate(np.ravel(axes)):
    ax.plot(ret[0], ret[1][i], '.', alpha=0.2)
    ax.set_xlabel("XGM pulse energy ($\mu$J)")
    ax.set_ylabel("{} integration (arb.)".format(adc_channel_names[i]))

fig.suptitle(run_folder, fontsize=16)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])